# [Advent of Code 2020 Day 13](https://adventofcode.com/2020/day/13)

More simulation perhaps? The `x`s scare me though. I am going to guess part 2 will involve simulating with bus lines would be optimal vs. part 1.

## Initial setup

In [79]:
import ipytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()

## Input Parsing

In [80]:
def parse_input(filename: str) -> Any:

    gen = yield_line(filename)

    earliest_time = int(next(gen))
    ids = next(gen).split(",")

    return earliest_time, ids

## Part 1
Lorem ipsum

In [81]:
def part_one(data: Any) -> int | str:
    earliest_time, ids = data
    best_time = math.inf
    best_bus = math.inf
    for bus_id in ids:
        if bus_id == "x":
            continue
        bus = int(bus_id)
        curr_time = 0
        while curr_time < earliest_time:
            curr_time += bus
        if curr_time < best_time:
            best_time = curr_time
            best_bus = bus
    return (best_time - earliest_time) * best_bus

In [82]:
%%ipytest
def test_part_one():
    assert part_one(parse_input("example1")) == 295
    assert part_one(parse_input("input")) == 138

.                                                                                            [100%]
1 passed in 0.02s


## Part 2
Lorem ipsum

In [83]:
def part_two(data: Any) -> int | str:
    return 0x3f3f3f3f + 2

In [84]:
%%ipytest
def test_part_two():
    assert part_two(parse_input("example1")) == 0x3f3f3f3f + 2
    assert part_two(parse_input("input")) == 0x3f3f3f3f + 2

.                                                                                            [100%]
1 passed in 0.01s
